In [15]:
pip freeze > model_requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd

In [17]:
data_train_df = pd.read_csv("data_train_min_max_scaled_csv")
data_test_df = pd.read_csv("data_test_csv")

In [18]:
data_train_df = data_train_df.drop(['Unnamed: 0', 'Row ID'], axis=1)
data_test_df = data_test_df.drop(['Unnamed: 0', 'Row ID'], axis=1)

In [19]:
data_train_df = data_train_df.drop(['Start Team Value', 'Remaining overs and balls', 'Remaining Average Team Value', 'Winner_num'], axis=1)
y_train = data_train_df['Winner_num']
X_train = data_train_df.drop('Winner_num',axis=1)
X_test = data_test_df.drop(['Start Team Value', 'Remaining overs and balls', 'Remaining Average Team Value', 'Winner_num'],axis=1), 
y_test = data_test_df['Winner_num']

In [20]:
data_train_df

,Wickets taken,Remaining Team Value,Ground RPO,Ground RPW,Winner_num,Remainder
0,0.156259,0.071429,-0.348593,-0.259181,1,0.990530
1,0.156259,0.071429,-0.348593,-0.259181,1,0.988617
2,0.156259,0.071429,-0.348593,-0.259181,1,0.984773
3,0.156259,0.071429,-0.348593,-0.259181,1,0.982841
4,0.156259,0.071429,-0.348593,-0.259181,1,0.971114
...,...,...,...,...,...,...
216173,0.156259,-0.142857,0.084371,-0.098113,2,-0.164188
216174,0.156259,-0.142857,0.084371,-0.098113,2,-0.192604
216175,0.156259,-0.142857,0.084371,-0.098113,2,-0.270374
216176,0.156259,-0.142857,0.084371,-0.098113,2,-0.287257


In [21]:
data_test_df

,Wickets taken,Remaining Team Value,Remaining Average Team Value,Start Team Value,Ground RPO,Ground RPW,Winner_num,Remaining overs and balls,Remainder
0,3,16,2.000000,24,4.79,29.31,1,177,29.3
1,3,16,2.000000,24,4.79,29.31,1,174,29.0
2,3,16,2.000000,24,4.79,29.31,1,167,27.5
3,3,16,2.000000,24,4.79,29.31,1,163,27.1
4,3,16,2.000000,24,4.79,29.31,1,156,26.0
...,...,...,...,...,...,...,...,...,...
54040,2,16,1.777778,21,5.40,31.64,2,38,6.2
54041,2,16,1.777778,21,5.40,31.64,2,36,6.0
54042,3,13,1.625000,21,5.40,31.64,2,29,4.5
54043,3,13,1.625000,21,5.40,31.64,2,19,3.1


In [22]:
#y_train = data_train_df['Winner_num']
#X_train = data_train_df.drop('Winner_num',axis=1)
#X_test = data_test_df.drop('Winner_num',axis=1)
#y_test = data_test_df['Winner_num']

In [23]:
data_train_df

,Wickets taken,Remaining Team Value,Ground RPO,Ground RPW,Winner_num,Remainder
0,0.156259,0.071429,-0.348593,-0.259181,1,0.990530
1,0.156259,0.071429,-0.348593,-0.259181,1,0.988617
2,0.156259,0.071429,-0.348593,-0.259181,1,0.984773
3,0.156259,0.071429,-0.348593,-0.259181,1,0.982841
4,0.156259,0.071429,-0.348593,-0.259181,1,0.971114
...,...,...,...,...,...,...
216173,0.156259,-0.142857,0.084371,-0.098113,2,-0.164188
216174,0.156259,-0.142857,0.084371,-0.098113,2,-0.192604
216175,0.156259,-0.142857,0.084371,-0.098113,2,-0.270374
216176,0.156259,-0.142857,0.084371,-0.098113,2,-0.287257


Random Forest model

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
X_train

,Wickets taken,Remaining Team Value,Ground RPO,Ground RPW,Remainder
0,0.156259,0.071429,-0.348593,-0.259181,0.990530
1,0.156259,0.071429,-0.348593,-0.259181,0.988617
2,0.156259,0.071429,-0.348593,-0.259181,0.984773
3,0.156259,0.071429,-0.348593,-0.259181,0.982841
4,0.156259,0.071429,-0.348593,-0.259181,0.971114
...,...,...,...,...,...
216173,0.156259,-0.142857,0.084371,-0.098113,-0.164188
216174,0.156259,-0.142857,0.084371,-0.098113,-0.192604
216175,0.156259,-0.142857,0.084371,-0.098113,-0.270374
216176,0.156259,-0.142857,0.084371,-0.098113,-0.287257


In [28]:
X_test

,Wickets taken,Remaining Team Value,Remaining Average Team Value,Start Team Value,Ground RPO,Ground RPW,Remaining overs and balls,Remainder
0,3,16,2.000000,24,4.79,29.31,177,29.3
1,3,16,2.000000,24,4.79,29.31,174,29.0
2,3,16,2.000000,24,4.79,29.31,167,27.5
3,3,16,2.000000,24,4.79,29.31,163,27.1
4,3,16,2.000000,24,4.79,29.31,156,26.0
...,...,...,...,...,...,...,...,...
54040,2,16,1.777778,21,5.40,31.64,38,6.2
54041,2,16,1.777778,21,5.40,31.64,36,6.0
54042,3,13,1.625000,21,5.40,31.64,29,4.5
54043,3,13,1.625000,21,5.40,31.64,19,3.1


In [26]:
rnd_clf = RandomForestClassifier(random_state=7)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Remaining Average Team Value
- Remaining overs and balls
- Start Team Value


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay

accuracy = accuracy_score(y_test, y_pred_rf)
print("Accuracy:", accuracy)

In [ ]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV, train_test_split


param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)

In [ ]:
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=379, max_depth=18,
                                 n_jobs=-1, random_state=7)
rnd_clf.fit(X_train, y_train)

y_tweaked_pred_rf = rnd_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay

accuracy = accuracy_score(y_test, y_tweaked_pred_rf)
print("Accuracy:", accuracy)

In [ ]:
# Create a series containing feature importances from the model and feature names from the training data
feature_importances = pd.Series(best_rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

# Plot a simple bar chart
feature_importances.plot.bar();

https://www.datacamp.com/tutorial/random-forests-classifier-python

Neural Network

In [ ]:
!pip install keras
!pip install tqdm
!pip3 install tensorflow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm import tqdm

from sklearn.preprocessing import OneHotEncoder

In [ ]:
pip install ipywidgets

In [ ]:
%pip install --upgrade jupyter ipywidgets
%jupyter nbextension enable --py widgetsnbextension # removed !pip on the recommendation of a comment.

import time
from tqdm.notebook import tqdm

In [ ]:
pip install jupyter

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
features = ['Wickets taken', 'Remaining Team Value', 'Remaining Average Team Value', 'Start Team Value', 'Ground RPO', 'Ground RPW','Winner_num', 'Remaining overs and balls', 'Remainder']

In [ ]:
# first neural network with keras tutorial
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_shape=(8,), activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_shape=(8,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=150, batch_size=10)


In [ ]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
#1689/1689 [==============================] - 7s 4ms/step - loss: -22466.3770 - accuracy: 0.5082
#Accuracy: 50.82

https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

SVM model

In [ ]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))